In [ ]:
pip install git+https://github.com/openai/swarm.git

In [ ]:
!pip install --upgrade swarm
!pip install google-search-results
!pip install python-dotenv

In [ ]:
# Shared trip plan dictionary
trip_plan = {}


In [ ]:
results

import os
from serpapi import GoogleSearch
import pprint


In [ ]:
from serpapi import GoogleSearch
import json
import pprint

# Replace with your actual SerpAPI API key
SERPAPI_API_KEY = ''  # 🔑 Replace this with your SerpAPI API key

# --------------------------
# Flight Search Parameters
# --------------------------
flight_params = {
    "engine": "google_flights",
    "departure_id": "MIA",         # Departure Airport ID (e.g., Miami International Airport)
    "arrival_id": "CDG",           # Arrival Airport ID (e.g., Charles de Gaulle Airport)
    "outbound_date": "2025-06-22",
    "return_date": "2025-06-30",
    "currency": "USD",
    "hl": "en",
    "type": "1",                   # Round trip
    "travel_class": "4",           # Travel class (Assuming '4' corresponds to a specific class)
    "multi_city_json": json.dumps([
        {"departure_id": "CDG", "arrival_id": "LAX", "date": "2024-11-06"},
        {"departure_id": "LAX", "arrival_id": "CDG", "date": "2024-11-12"}
    ]),
    "api_key": SERPAPI_API_KEY
}

# Initialize and execute flight search
flight_search = GoogleSearch(flight_params)
flight_results = flight_search.get_dict()

# Optional: Print the first flight option for verification
print("\n--- First Flight Option ---\n")
pprint.pprint(flight_results.get("best_flights", [])[:1])

# --------------------------
# Hotel Search Parameters
# --------------------------
hotel_params = {
    "engine": "google_hotels",
    "q": "Hotels in Paris",
    "check_in_date": "2025-06-23",
    "check_out_date": "2025-06-30",
    "adults": "1",
    "children": "0",
    "hotel_class": "5",       # Five-star hotels
    "currency": "USD",        # Currency preference
    "hl": "en",               # Language code (optional)
    "gl": "us",               # Country code (optional)
    "api_key": SERPAPI_API_KEY
}

# Initialize and execute hotel search
hotel_search = GoogleSearch(hotel_params)
hotel_results = hotel_search.get_dict()

# Pretty-print the entire hotel response for debugging purposes
print("\n--- Full Hotel API Response ---\n")
pprint.pprint(hotel_results)

# Extract the list of hotels
properties = hotel_results.get("properties", [])

# Check if any hotels were found
if not properties:
    print("\n🚫 No hotels found for the given criteria.")
else:
    print(f"\n✅ Found {len(properties)} hotel(s) in Paris.\n")

    # Sort the hotels by 'extracted_lowest' price in descending order (most expensive first)
    sorted_hotels = sorted(
        properties,
        key=lambda x: x.get('rate_per_night', {}).get('extracted_lowest', 0),
        reverse=True
    )

    # Debug: Print sorted hotel names with their prices
    print("🔽 Hotels sorted by price (most expensive first):\n")
    for idx, hotel in enumerate(sorted_hotels, start=1):
        price = hotel.get('rate_per_night', {}).get('extracted_lowest', 'N/A')
        print(f"{idx}. {hotel.get('name', 'N/A')} - ${price}")
    print("\n")

    # Select the third most expensive hotel
    try:
        third_hotel = sorted_hotels[2]  # Indexing starts at 0
        print("--- 🏨 Third Most Expensive Hotel ---")
        print(f"Name: {third_hotel.get('name', 'N/A')}")
        print(f"Address: {third_hotel.get('address', 'N/A')}")
        print(f"Description: {third_hotel.get('description', 'N/A')}")
        print(f"Rating: {third_hotel.get('overall_rating', 'N/A')}")
        print(f"Reviews: {third_hotel.get('reviews', 'N/A')}")
        print(f"Price per Night: ${third_hotel.get('rate_per_night', {}).get('lowest', 'N/A')}")
        print(f"Booking Link: {third_hotel.get('link') or third_hotel.get('booking_link') or 'N/A'}")
        print(f"Amenities: {', '.join(third_hotel.get('amenities', []))}")
        print("\n")

        # Assign flight_results and selected_hotel to variables for agents
        flight_data = flight_results
        selected_hotel = third_hotel

    except IndexError:
        print("⚠️ Less than three hotels found. Unable to select the third most expensive hotel.")
        # Assign flight_results and set selected_hotel as None
        flight_data = flight_results
        selected_hotel = None

# --------------------------
# Prepare Data for Agents
# --------------------------
# You can structure the data as needed for your agents. For example:
trip_data = {
    "flights": flight_data,
    "hotel": selected_hotel
}

# This 'trip_data' dictionary can be passed to agents in the next cell or process


In [ ]:
import os

# Set your OpenAI API key as an environment variable
os.environ['OPENAI_API_KEY'] = ''

In [ ]:
from swarm import Swarm, Agent  # Ensure you have the correct import

# Initialize the Swarm client
client = Swarm()

# Define transfer functions for agents
def transfer_to_agent_b():
    return agent_b

def transfer_to_agent_c():
    return agent_c

def transfer_to_agent_d():
    return agent_d

# Define Agent D: Organize information day by day
agent_d = Agent(
    name="Agent D",
    instructions=(
        "You are a trip planner. "
        "Receive information as a prompt, and organize the attractions and restaurant bookings day by day for the user's trip to Paris. "
        "They arrive on a Monday and their last night in the city is a Sunday night."
    ),
    # No functions needed for Agent D
)

# Define Agent B: Find attractions in Paris
agent_b = Agent(
    name="Agent B",
    instructions=(
        "You are an expert in Parisian attractions. "
        "Provide a list of must-see attractions and activities to do in Paris. "
        "Plan for one activity in the AM and one in the PM each day."
    ),
    functions=[transfer_to_agent_d],  # Pass the function without calling it
)

# Define Agent C: Find top restaurants in Paris
agent_c = Agent(
    name="Agent C",
    instructions=(
        "You are a top-tier restaurant finding agent. "
        "Provide recommendations for the greatest restaurants on planet Earth for dinner in Paris."
    ),
    functions=[transfer_to_agent_d],  # Pass the function without calling it
    # If Agent C doesn't need to delegate further, you can omit the functions or leave it empty
)

# Define Agent A: Main coordinator
agent_a = Agent(
    name="Agent A",
    instructions=(
        "You are the main trip planning coordinator. "
        "When a user requests a trip plan to Paris, delegate tasks to Agent B and Agent C, "
        "collect their responses, pass the collected information to Agent D for organization, "
        "and then present the organized trip plan to the user."
    ),
    functions=[transfer_to_agent_b, transfer_to_agent_c],  # Only delegate to B and C
)

# User request message
user_message = {"role": "user", "content": "Plan my trip to Paris."}

# Run the swarm
response = client.run(
    agent=agent_d,
    messages=[user_message],
)

# Print the organized trip plan
print(response.messages[-1]["content"])
